In [1]:
import torch
import torch.nn as nn
from torch.nn import Embedding

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import dataset
import test_lstm

2023-03-08 00:02:55.358512: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
lie_trial_path = './processed_lie/' #60 entries
truth_trial_path = './processed_truth/' #61 entries
MU3D_path = './processed/' # 300 entries

In [11]:
# no split by person
X, Y = dataset.preprocessing(truth_trial_path, lie_trial_path, numOfFrames=10)

TEST_RATIO = 0.2

xTrain, xTest = train_test_split(X, test_size=TEST_RATIO, shuffle=False)
yTrain, yTest = train_test_split(Y, test_size=TEST_RATIO, shuffle=False)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
embedding_layer = Embedding(num_embeddings=10, embedding_dim=4)

positions = torch.tensor([0,1,2,3,4,5,6,7,8,9])
embedded_positions = embedding_layer(positions)
embedded_positions

tensor([[ 1.9189,  0.3183, -1.6296, -0.2299],
        [-0.1687, -1.0629,  1.4963,  0.7978],
        [-1.0853,  0.2260,  0.1388,  1.1185],
        [-0.3525, -0.6287,  0.4615, -0.6831],
        [ 1.4058, -0.7720, -1.6984,  0.7521],
        [ 0.6558,  0.5897, -0.1031,  1.9207],
        [-0.7116,  0.8319,  0.7790, -0.1593],
        [ 1.3356, -0.0947, -1.7079,  0.2963],
        [-0.1453,  0.9592,  0.0719,  0.6384],
        [ 1.1880,  1.3794, -0.6667, -1.1403]], grad_fn=<EmbeddingBackward0>)

In [6]:
#embedding
def gen_loc_list(max_frames_count, pos_encoder_size, frames_count = 10):
  embedding = nn.Embedding(max_frames_count, pos_encoder_size)
  input = torch.tensor([i for i in range(frames_count)]).clone().detach()
  loc_list = embedding(input)

  return loc_list

def gen_data(data_arr, embedding_arr, frames_count = 10):

  lst = []

  for idx in range(frames_count):
    data = data_arr[idx].clone().detach()
    embedding = embedding_arr[idx].clone().detach()
    lst.append(torch.cat((data, embedding), 0))

  return lst

In [7]:
#model

from torch.nn import TransformerEncoder, TransformerEncoderLayer

class TransformerModel(nn.Module):

  def __init__(self, inFeatCount, num_encoder_layers, n_heads = 5, n_hidden = 500, dropout = 0.3, outFeatCount = 2): #Out = [prob for 0, prob for 1]
    super(TransformerModel, self).__init__()

    encoder_layer = TransformerEncoderLayer(inFeatCount, n_heads, n_hidden, dropout)

    self.encoder = TransformerEncoder(encoder_layer, num_encoder_layers)

    self.decoder = nn.Linear(inFeatCount, outFeatCount)

  def init_weights(self):
      initrange = 0.1
      self.encoder.weight.data.uniform_(-initrange, initrange)
      self.decoder.bias.data.zero_()
      self.decoder.weight.data.uniform_(-initrange, initrange)

  def forward(self, data):
      encoded = self.encoder(data)
      output = self.decoder(encoded)
      return torch.nn.functional.softmax(output)
  
def predict(model, inputArr):
    inputArr = torch.from_numpy(inputArr).float()
    inputArr = inputArr.to(device)
    
    output = model(inputArr)
    
    return output


In [8]:
max_frames_count = 20
pos_encoder_size = 5
data_arr = xTrain
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
#generate embedding
embedding_arr = gen_loc_list(max_frames_count, pos_encoder_size)

#generate embedded data
data = gen_data(data_arr[0], embedding_arr)

#generate model
model = TransformerModel(10, 5)

arr = np.array([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])
predict(model, arr)


/var/folders/66/pn8653k51_75x5ksbpsy8hm80000gn/T/ipykernel_1575/2287887436.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return torch.nn.functional.softmax(output)


tensor([[0.3312, 0.6688],
        [0.4603, 0.5397],
        [0.3704, 0.6296],
        [0.3317, 0.6683],
        [0.4101, 0.5899]], grad_fn=<SoftmaxBackward0>)

In [12]:
# training
import torch.optim as optim

def train(model, xTrain, yTrain, xTest, yTest, epochs = 100, lr = 0.001, batch_size = 32):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr = lr)
    
    train_losses = []
    test_losses = []
    train_acc = []
    test_acc = []
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        print(xTrain.shape)
    
        result = predict(model, xTrain)
        loss = criterion(result, yTrain)
        loss.backward()
        optimizer.step()

        print(xTrain.shape)
    
        train_losses.append(loss.item())
    
        model.eval()
    
        print("Epoch: ", epoch, " Train Loss: ", train_losses[-1], " Test Loss: ", test_losses[-1], " Train Acc: ", train_acc[-1], " Test Acc: ", test_acc[-1])
        optimizer.step()
    
    return train_losses, test_losses, train_acc, test_acc

train(model, xTrain, yTrain, xTest, yTest)

(49150, 10, 10)


: 

: 